## IMAGE CLASSIFICATION USING CIFAR-10 DATASET: A COMPARISON OF MANHATTAN (L1) AND EUCLIDEAN (L2) DISTANCES WITH 5-FOLD CROSS-VALIDATION
***
#### **Name:** Foysal Ahamed Nirob
#### **Id:** 19-40925-2
#### **Section:** CVPR[C]







## **Introduction:**

This assignment explores the classification of images using the CIFAR-10 dataset. The dataset consists of 60,000 32*32*3 color images in 10 categories. Catagories are: Airplane, Automobile, Bird, Cat, Deer, Dog, Frog, Horse, Ship, Truck. The objective is to create an image classification model and compare the Manhattan (L1) and Euclidean (L2) distance calculation methods. The model's performance will be evaluated using 5-fold cross-validation. The training data, consisting of 50,000 images, will be transformed into grayscale (32x32) to optimize computational efficiency. Additionally, a graphical representation will be generated to visualize the accuracy of different hyperparameter values (K) or number of neighbors for both distance calculation techniques. Additionally, the discussion will focus on determining the more suitable distance calculation method for this dataset, and the top 5 predictions made by the model will be presented.


## **Dataset and Preprocessing:**

We will be working with the CIFAR-10 dataset, which is widely used for image classification tasks. The CIFAR-10 dataset consists of 60,000 32x32x3 color images in ten different categories or classes.

To begin, you can download the CIFAR-10 dataset from the following link: __[CIFAR-10 Dataset](https://github.com/YoongiKim/CIFAR-10-images)__

Once you have obtained the dataset, we will proceed with the preprocessing steps. Specifically, we will focus on converting the 50,000 training images to grayscale, while keeping their dimensions at 32x32 pixels. Ch